In [1]:
import numpy as np
import tensorflow as tf
import scipy.special as ss

In [2]:
a = tf.convert_to_tensor(np.arange(10))

2023-02-12 11:30:08.301304: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 11:30:08.303692: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
a

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [4]:
a > 5

<tf.Tensor: shape=(10,), dtype=bool, numpy=
array([False, False, False, False, False, False,  True,  True,  True,
        True])>

In [18]:
labels = tf.cast(a > 5, tf.float32)

In [10]:
probs = np.linspace(0.1,0.9, len(labels))

In [9]:
bc = tf.keras.losses.BinaryCrossentropy()

In [10]:
bc.name

'binary_crossentropy'

In [23]:
bc.call(tf.cast(labels,tf.float32), tf.cast(probs,tf.float32))

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)>

In [14]:
import sys
sys.path.append('../')
from general_purpose import utilities as ut

In [24]:
ut.entropy(tf.cast(labels, tf.float32),probs)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.10536052, 0.20935023, 0.3254224 , 0.4567584 , 0.6079894 ,
       0.7862376 , 0.4567584 , 0.3254224 , 0.20935023, 0.10536052],
      dtype=float32)>

In [27]:
print(tf.keras.losses.Loss.__doc__)

Loss base class.

  To be implemented by subclasses:
  * `call()`: Contains the logic for loss calculation using `y_true`, `y_pred`.

  Example subclass implementation:

  ```python
  class MeanSquaredError(Loss):

    def call(self, y_true, y_pred):
      y_pred = tf.convert_to_tensor_v2(y_pred)
      y_true = tf.cast(y_true, y_pred.dtype)
      return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
  ```

  When used with `tf.distribute.Strategy`, outside of built-in training loops
  such as `tf.keras` `compile` and `fit`, please use 'SUM' or 'NONE' reduction
  types, and reduce losses explicitly in your training loop. Using 'AUTO' or
  'SUM_OVER_BATCH_SIZE' will raise an error.

  Please see this custom training [tutorial](
    https://www.tensorflow.org/tutorials/distribute/custom_training) for more
  details on this.

  You can implement 'SUM_OVER_BATCH_SIZE' using global batch size like:

  ```python
  with strategy.scope():
    loss_obj = tf.keras.losses.CategoricalCro

In [2]:
class StupidLoss(tf.keras.losses.Loss):
    def __init__(self, name=None):
        super().__init__(name=name or self.__class__.__name__)

    def call(self, y_true, y_pred):
        return tf.math.square(y_true - y_pred)

In [15]:
cls = StupidLoss
ss = cls()
ss.name

'StupidLoss'

In [3]:
s = StupidLoss()

In [4]:
s.name

'StupidLoss'

In [5]:
def weighted(cls, function):
    class WeightedLoss(cls):
        def __init__(self, name=f'Weighted{cls.__name__}'):
            super().__init__(name=name or self.__class__.__name__)
            self.function = function

        def call(self, y_true, y_pred):
            weights = self.function(y_true)
            loss = super().call(y_true, y_pred)
            assert weights.shape == loss.shape
            return weights*loss
        
    return WeightedLoss

In [11]:
s = StupidLoss()
ws = weighted(StupidLoss, lambda x: x)()

In [8]:
ws.__class__.__name__

'WeightedLoss'

In [10]:
y_true = tf.convert_to_tensor(np.arange(0,1,0.1))
y_pred = y_true + np.random.uniform(0,1, size=y_true.shape)
y_true, y_pred

(<tf.Tensor: shape=(10,), dtype=float64, numpy=array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])>,
 <tf.Tensor: shape=(10,), dtype=float64, numpy=
 array([0.91087622, 0.89300822, 0.64845848, 0.96271565, 1.20611952,
        1.00231339, 1.19514641, 0.70717372, 0.92123473, 1.81214124])>)

In [12]:
s(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=0.4201962351799011>

In [13]:
ws(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=0.15940728783607483>

In [14]:
tf.reduce_mean(y_true*s.call(y_true, y_pred))

<tf.Tensor: shape=(), dtype=float64, numpy=0.15940729920612795>